In [1]:
import os, random
import numpy as np
import torch
from transformers import set_seed as hf_set_seed

SEED = 42

# стабильные словари/хеши Python
os.environ["PYTHONHASHSEED"] = str(SEED)

# детерминизм в линейной алгебре (PyTorch>=1.8/2.x)
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":16:8"   # или ":4096:8" на некоторых GPU

# отключаем подбор «самого быстрого» алгоритма cuDNN (он недетерминирует)
torch.backends.cudnn.benchmark = False

# включаем строго детерминированные алгоритмы (может замедлить обучение)
try:
    torch.use_deterministic_algorithms(True)
except Exception:
    # для старых версий PyTorch
    torch.backends.cudnn.deterministic = True

# сидим всё подряд
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
hf_set_seed(SEED)  # сидит random/numpy/torch; есть флаг deterministic у новых версий


c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from deeppavlov import train_model, build_model 
from deeppavlov.core.commands.utils import parse_config

PROJECT_DIR = '..'
MODEL_NAME = 'model'

model_config = parse_config('ner_collection3_bert')

# dataset that the model was trained on
print(model_config['dataset_reader']['data_path'])

model_config['dataset_reader']['data_path'] = PROJECT_DIR + '/datasets/conll/'

del model_config['metadata']['download']


model_config['dataset_reader']['iobes'] = False
model_config['metadata']['variables']['MODEL_PATH'] = PROJECT_DIR + '/models/' + MODEL_NAME

model_config['chainer']['pipe'][1]['save_path'] = PROJECT_DIR + '/models/tag.dict'
model_config['chainer']['pipe'][1]['load_path'] = PROJECT_DIR + '/models/tag.dict'

model_config['chainer']['pipe'][2]['save_path'] = PROJECT_DIR + '/models/' + MODEL_NAME
model_config['chainer']['pipe'][2]['load_path'] = PROJECT_DIR + '/models/' + MODEL_NAME
model_config['chainer']['pipe'][2]['learning_rate_drop_patience'] = 20


model_config['dataset_iterator'] = {
  "class_name": "data_learning_iterator",
  "seed": 42,
  "shuffle": True
}

model_config['train']['batch_size'] = 550

model_config['train']['log_every_n_batches'] = 10
model_config['train']['val_every_n_batches'] = 10



~/.deeppavlov/downloads/collection3/


In [3]:
model_config

{'dataset_reader': {'class_name': 'conll2003_reader',
  'data_path': '../datasets/conll/',
  'dataset_name': 'collection3',
  'provide_pos': False,
  'provide_chunk': False,
  'iobes': False},
 'dataset_iterator': {'class_name': 'data_learning_iterator',
  'seed': 42,
  'shuffle': True},
 'chainer': {'in': ['x'],
  'in_y': ['y'],
  'pipe': [{'class_name': 'torch_transformers_ner_preprocessor',
    'vocab_file': 'DeepPavlov/rubert-base-cased',
    'do_lower_case': False,
    'max_seq_length': 512,
    'max_subword_length': 15,
    'token_masking_prob': 0.0,
    'in': ['x'],
    'out': ['x_tokens',
     'x_subword_tokens',
     'x_subword_tok_ids',
     'startofword_markers',
     'attention_mask',
     'tokens_offsets']},
   {'id': 'tag_vocab',
    'class_name': 'simple_vocab',
    'unk_token': ['O'],
    'pad_with_zeros': True,
    'save_path': '../models/tag.dict',
    'load_path': '../models/tag.dict',
    'fit_on': ['y'],
    'in': ['y'],
    'out': ['y_ind']},
   {'class_name': 'to

In [4]:

ner_model = train_model(model_config, download=True)

2025-10-01 12:03:56.757 WARNING in 'deeppavlov.core.trainers.fit_trainer'['fit_trainer'] at line 66: TorchTrainer got additional init parameters ['pytest_max_batches', 'pytest_batch_size'] that will be ignored:
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
2025-10-01 12:03:58.939 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 104: [saving vocabulary to C:\Users\lexan\OneDrive\Documents\hackaton_lct\models\tag.dict]
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.pr

{"valid": {"eval_examples_count": 4761, "metrics": {"ner_f1": 26.3928, "ner_token_f1": 39.0729}, "time_spent": "0:00:02", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 100}}


1it [00:00,  6.57it/s]

{"train": {"eval_examples_count": 550, "metrics": {"ner_f1": 37.5357, "ner_token_f1": 42.5933}, "time_spent": "0:00:06", "epochs_done": 0, "batches_seen": 10, "train_examples_seen": 5500, "loss": 1.1858988523483276}}



9it [00:01,  8.53it/s]
2025-10-01 12:04:06.722 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 206: Improved best ner_f1 from 26.3928 to 58.1043
2025-10-01 12:04:06.723 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 208: Saving model
2025-10-01 12:04:06.724 INFO in 'deeppavlov.core.models.torch_model'['torch_model'] at line 175: Saving model to C:\Users\lexan\OneDrive\Documents\hackaton_lct\models\model.pth.tar.


...

48it [01:30,  1.89s/it]
2025-10-01 12:16:23.325 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 336: Stopped training
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a Bert

{"valid": {"eval_examples_count": 4761, "metrics": {"ner_f1": 93.2843, "ner_token_f1": 94.7687}, "time_spent": "0:00:03"}}



1it [00:00,  3.97it/s]

{"test": {"eval_examples_count": 529, "metrics": {"ner_f1": 91.3658, "ner_token_f1": 91.8587}, "time_spent": "0:00:01"}}



Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initiali

In [10]:
model_config

{'dataset_reader': {'class_name': 'conll2003_reader',
  'data_path': '../datasets/conll/',
  'dataset_name': 'collection3',
  'provide_pos': False,
  'provide_chunk': False,
  'iobes': True},
 'dataset_iterator': {'class_name': 'data_learning_iterator'},
 'chainer': {'in': ['x'],
  'in_y': ['y'],
  'pipe': [{'class_name': 'torch_transformers_ner_preprocessor',
    'vocab_file': 'DeepPavlov/rubert-base-cased',
    'do_lower_case': False,
    'max_seq_length': 512,
    'max_subword_length': 15,
    'token_masking_prob': 0.0,
    'in': ['x'],
    'out': ['x_tokens',
     'x_subword_tokens',
     'x_subword_tok_ids',
     'startofword_markers',
     'attention_mask',
     'tokens_offsets']},
   {'id': 'tag_vocab',
    'class_name': 'simple_vocab',
    'unk_token': ['O'],
    'pad_with_zeros': True,
    'save_path': '../models/tag.dict',
    'load_path': '../models/tag.dict',
    'fit_on': ['y'],
    'in': ['y'],
    'out': ['y_ind']},
   {'class_name': 'torch_transformers_sequence_tagger',

In [3]:
from deeppavlov import build_model

ner_model = build_model(model_config, download=False)

c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\lexan\miniconda3\envs\dp310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNo

In [19]:
ner_model(['artfruit нектари'])

[[['artfruit', 'нектари']], [['S-BRAND', 'S-TYPE']]]